In [1]:
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH, THEMES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Centering and fixing title
def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Adding custom colorscales (to add one: themes/custom_colorscales.yaml)
import yaml
custom_colorscales = yaml.load(open(THEMES_PATH / 'custom_colorscales.yaml', 'r'))
cf.colors._custom_scales['qual'].update(custom_colorscales)
cf.colors.reset_scales()

# Setting cuffilinks template (use it with .iplot(theme='custom')
cf.themes.THEMES['custom'] = yaml.load(open(THEMES_PATH / 'cufflinks_template.yaml', 'r'))

# Carrega coisas

### Config

In [2]:
config = yaml.load(open('../src/configs/config.yaml', 'r'))

### Capacidade Hospitalar dos municípios

In [7]:
from src import loader

In [8]:
cities = loader._read_cities_data('br', config)
cities = pd.merge(
    cities['cities_population'], cities['health_infrastructure'],
    on='city_id', how='left', suffixes=('', '_y'))
cities = cities.drop([c for c in cities.columns if '_y' in c], 1)

### Histórico de Todos os Casos

In [9]:
cases = pd.read_csv('https://data.brasil.io/dataset/covid19/caso_full.csv.gz')

In [36]:
cases[cases['city_ibge_code'] == 3550308.0].sort_values(by='date')\
.set_index('date')['last_available_confirmed'].iplot(
    title='Evolução de Casos em São Paulo',
    yTitle='Número de casos',
    theme='custom', width=5
)

### Últimos casos

In [17]:
last_cases = src.loader._read_cases_data('br', config)

In [18]:
last_cases

,city_id,number_cases,last_updated,deaths
0,3101607,1,2020-04-11,0.0
1,3101805,1,2020-04-11,0.0
2,3103504,5,2020-04-11,0.0
3,3104205,2,2020-04-11,0.0
4,3105608,2,2020-04-11,0.0
...,...,...,...,...
942,2923308,0,2020-04-06,0.0
943,3305901,0,2020-04-06,0.0
944,2300101,1,2020-04-04,0.0
945,2306306,1,2020-04-04,0.0


# Modelo

In [20]:
from src.model import simulator

### Carrega Parâmetros

In [21]:
params = dict()

In [22]:
params['population'] = {
     'N': 100,
     'I': 1,
     'D': 0,
     'R': 0
}

In [23]:
params['strategy'] =  {
        'isolation': 1,
        'lockdown': 90}

In [24]:
params['supply'] = {
    'n_beds': 9,
    'n_ventilators': 1
}

### Roda Modelo

In [25]:
dfs = simulator.run_simulation(params['population'], params['strategy'], config)

In [31]:
dfs['worst'].head()

,S,E,I1,I2,I3,R,D,N,scenario
dias,,,,,,,,,
1,97.310768,1.434521,0.959184,0.121715,0.025322,0.145974,0.002516,100.0,nothing
2,96.662399,1.763384,1.107072,0.125996,0.025687,0.310396,0.005066,100.0,nothing
3,95.920102,2.118099,1.295029,0.133374,0.026167,0.499573,0.007657,100.0,nothing
4,95.062323,2.513501,1.523128,0.144259,0.026837,0.719646,0.010306,100.0,nothing
5,94.067803,2.961488,1.793863,0.159032,0.027774,0.977005,0.013034,100.0,nothing


### Calcula dday

In [26]:
simulator.get_dday(dfs, 'I2', params['supply']['n_beds'])

{'worst': -1, 'best': -1}